In [10]:
import os
import tweepy as tw

In [11]:
#removed my private api keys

ACCESS_TOKEN = ''
ACCESS_SECRET = ''
CONSUMER_KEY = ''
CONSUMER_SECRET = ''

In [12]:
auth = tw.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
auth.set_access_token(ACCESS_TOKEN, ACCESS_SECRET)
api = tw.API(auth,wait_on_rate_limit=True)

In [13]:
keywords = [ "Canada", "University", "Dalhousie University", "Halifax","Canada Education"]
date_since = "2018-03-01"

In [15]:
all_tweet = []

for i in keywords[2:3]:
    if i == "Canada":
        tweets = tw.Cursor(api.search,
                      q=i,
                      lang="en",
                      since=date_since,
                      tweet_mode='extended').items(1500)
    else:
        tweets = tw.Cursor(api.search,
                      q=i,
                      lang="en",
                      tweet_mode='extended').items(600)
    
    for tweet in tweets:
        t = {}
        t["hashtag"] = i
        t["text"] = tweet.full_text
        t["date_time"] = tweet.author.created_at
        t['auther_name'] = tweet.author.screen_name
        t['retweet_coount'] = tweet.retweet_count
        t['location'] = tweet.user.location
        t['source'] = tweet.source
        t['language'] = tweet.lang
        
        all_tweet.append(t)

In [2]:
#cleaning tweet function

def preprocess_tweet(text):
    
    # convert to lower case
    clean_str = clean_str.lower()
    
    # remove usernames
    clean_str = re.sub('@[^\s]+', '', clean_str)
    
    # remove # in #hashtag
    clean_str = re.sub(r'#([^\s]+)', r'\1', clean_str)
    
    # remove emojis
    clean_str = clean_str.encode('ascii', 'ignore').decode('ascii')
    
    # remove URLs
    clean_str = re.sub('((www\.[^\s]+)|(https?://[^\s]+)|(http?://[^\s]+))', '', clean_str)
    clean_str = re.sub(r'http\S+', '', clean_str)
    
    #remove \n
    clean_str = clean_str.replace('\n','')
    
    return str(clean_str.strip())

In [85]:
for each in tweet1:
    each['text'] = preprocess_tweet(each['text'])
    each['location'] = preprocess_tweet(each['location'])

In [6]:
#mongo connection 

from pymongo import MongoClient
client = MongoClient('localhost', 27017)

In [7]:
#db connection, and collection made

db = client[ "tweet_database"] 
tweet_collection = db['tweet_collection']

In [96]:
#inserted data to mongo collection

tweet_collection.insert_many(tweet1)

In [97]:
s = tweet_collection.find({})

In [104]:
import json

cursor = tweet_collection.find({})
file = open("tweet_collection1.json", "w")
file.write('[')
for document in cursor:
    file.write(json.dumps(document,default=str))
    file.write(',')
file.write(']')

1

In [107]:
# news API

In [109]:
!pip3 install newsapi-python

  Using cached https://files.pythonhosted.org/packages/1a/70/1935c770cb3be6e3a8b78ced23d7e0f3b187f5cbfab4749523ed65d7c9b1/requests-2.23.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/bc/a9/01ffebfb562e4274b6487b4bb1ddec7ca55ec7510b22e4c51f14098443b8/chardet-3.0.4-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/e8/74/6e4f91745020f967d09332bb2b8b9b10090957334692eb88ea4afe91b77f/urllib3-1.25.8-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/b9/63/df50cac98ea0d5b006c55a399c3bf1db9da7b5a24de7890bc9cfd5dd9e99/certifi-2019.11.28-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/89/e3/afebe61c546d18fb1709a61bee788254b40e736cff7271c7de5de2dc4128/idna-2.9-py2.py3-none-any.whl


In [4]:
from newsapi import NewsApiClient

In [5]:
newsapi = NewsApiClient(api_key='6d8b6998d3294f01a996a012f525c512')

In [6]:
keywords = ["Canada", "University", "Dalhousie University", "Halifax", "Canada Education", "Moncton", "Toronto"]

In [24]:
#fetching data realted to keywords from api

all_headlines = []

for each in keywords:
    top_headlines = newsapi.get_everything(q=each,sources=s)    

    for i in top_headlines['articles']:
        t = {}
        t['tag'] = "newsapi"
        t["hashtag"] = each
        t['source'] = i['source']['name']
        t['author'] = i['author']
        t['title'] = i['title']
        t['description'] = i['description']
        t['url'] = i['url']
        t['date'] = i['publishedAt']

        all_headlines.append(t)

In [61]:
#cleaning description 

for each in all_headlines:
    if each['description'] is not None:
        each['description'] = preprocess_tweet(each['description'])

In [8]:
db = client[ "tweet_database"] 
news_collection = db['news_collection']

In [65]:
news_collection.insert_many(all_headlines)

In [66]:
s = news_collection.find({})

In [67]:
for i in s:
    print(i)

{'_id': ObjectId('5e684063e8bdfb3a5f2f780d'), 'tag': 'newsapi', 'hashtag': 'Canada', 'source': 'Engadget', 'author': 'Christine Fisher', 'title': 'Starbucks embraces fake meat, starting in Canada', 'description': 'next week when starbucks rolls out its spring menu in stores across canada it will include one notable addition a breakfast sandwich with beyond meat sausage starbucks joins a growing list of brands that have added beyond meats plantbased offerings to t', 'url': 'https://www.engadget.com/2020/02/26/starbucks-beyond-meat-sandwich-canada/', 'date': '2020-02-26T19:35:00Z'}
{'_id': ObjectId('5e684063e8bdfb3a5f2f780e'), 'tag': 'newsapi', 'hashtag': 'Canada', 'source': 'BBC News', 'author': 'https://www.facebook.com/bbcnews', 'title': "Plans for first US 'safe injection site' derailed again", 'description': 'the addiction management method used in europe and canada has met determined opposition in the us', 'url': 'https://www.bbc.co.uk/news/world-us-canada-51669192', 'date': '2020-

In [4]:
# movie api
import requests

In [5]:
keywords = ["Canada", "University", "Moncton", "Halifax", "Toronto", "Vancouver", "Alberta", "Niagara"]

In [7]:
all_movie = []

for e in keywords:
    page = 1
    flag=True
    while flag:
        r = requests.get('http://www.omdbapi.com/?s={0}&apikey=cda6184e&page={1}'.format(e,page))
        print('http://www.omdbapi.com/?s={0}&apikey=cda6184e&page={1}'.format(e,page))
        page+=1
        data = r.json()        
        if data['Response'] == "True":

            for each in data['Search']:
                id = each['imdbID']
                
                r = requests.get('http://www.omdbapi.com/?i={}&apikey=15683061'.format(id))
                each = r.json()
                t = {}
                if each['Response'] == "True":
                    all_movie.append(each)
        else:
            flag = False

Canada
http://www.omdbapi.com/?s=Canada&apikey=cda6184e&page=1
http://www.omdbapi.com/?i=tt2776888&apikey=15683061
{'Title': 'Big Brother Canada', 'Year': '2013–', 'Rated': 'N/A', 'Released': '27 Feb 2013', 'Runtime': '60 min', 'Genre': 'Reality-TV', 'Director': 'N/A', 'Writer': 'N/A', 'Actors': 'Arisa Cox', 'Plot': 'The format for Big Brother in Canada remains largely unchanged from the U.S. edition, making them the only two version of the series thus far to follow this format. HouseGuests are ...', 'Language': 'English', 'Country': 'Canada', 'Awards': '1 win & 14 nominations.', 'Poster': 'https://m.media-amazon.com/images/M/MV5BZjY5MTUzYTUtNjA2MS00ZDUzLTgyMzUtNmQ2MTE2ZWJlMTNlXkEyXkFqcGdeQXVyMzQ2MDI5NjU@._V1_SX300.jpg', 'Ratings': [{'Source': 'Internet Movie Database', 'Value': '5.5/10'}], 'Metascore': 'N/A', 'imdbRating': '5.5', 'imdbVotes': '792', 'imdbID': 'tt2776888', 'Type': 'series', 'totalSeasons': '6', 'Response': 'True'}
http://www.omdbapi.com/?i=tt4116116&apikey=15683061
{'T

In [8]:
len(all_movie)

778

In [9]:
db = client[ "tweet_database"] 
movie_collection = db['movie_collection']

In [10]:
movie_collection.insert_many(all_movie)

In [11]:
s = movie_collection.find({})

In [12]:
for i in s:
    print(i)

{'_id': ObjectId('5e693d2f39e57a6146f77535'), 'Title': 'Toronto Mounted Rifles', 'Year': '1900', 'Rated': 'N/A', 'Released': '01 Feb 1900', 'Genre': 'Documentary, Short', 'Writer': 'N/A', 'Actors': 'N/A', 'Language': 'N/A', 'Country': 'USA', 'Awards': 'N/A', 'imdbRating': 'N/A', 'imdbVotes': 'N/A', 'Type': 'movie'}
{'_id': ObjectId('5e694085cda4379c70deb37a'), 'Title': 'Big Brother Canada', 'Year': '2013–', 'Rated': 'N/A', 'Released': '27 Feb 2013', 'Genre': 'Reality-TV', 'Writer': 'N/A', 'Actors': 'Arisa Cox', 'Language': 'English', 'Country': 'Canada', 'Awards': '1 win & 14 nominations.', 'imdbRating': '5.5', 'imdbVotes': '792', 'Type': 'series'}
{'_id': ObjectId('5e694085cda4379c70deb37b'), 'Title': 'My Internship in Canada', 'Year': '2015', 'Rated': 'N/A', 'Released': '26 May 2016', 'Genre': 'Comedy', 'Writer': 'Philippe Falardeau', 'Actors': 'Irdens Exantus, Caroline Trudel, Patrick Huard, Clémence Dufresne-Deslières', 'Language': 'French, English, Algonquin, Latin, Spanish', 'Cou

In [ ]:
# extracting imdbRating,Genre and Plot

extract_data = []

for each in s:
    t = {}
    t['rating'] = each['imdbRating']
    t['genre'] = each['Genre']
    t['plot'] = each['Plot']
    
    extract_data.append(t)
    
print(extract_data)

In [1]:
#spark

In [10]:
# ! pip3 install findspark 
! pip3 --no-cache-dir install pyspark

    100% |████████████████████████████████| 217.8MB 43.6MB/s ta 0:00:01 2% |▉                               | 5.5MB 29.8MB/s eta 0:00:08    6% |██                              | 13.7MB 25.1MB/s eta 0:00:09    9% |███                             | 20.2MB 22.4MB/s eta 0:00:09    11% |███▊                            | 25.2MB 32.2MB/s eta 0:00:06    12% |████                            | 26.9MB 32.7MB/s eta 0:00:06    16% |█████▍                          | 36.9MB 32.0MB/s eta 0:00:06    18% |█████▉                          | 39.5MB 19.5MB/s eta 0:00:10    20% |██████▍                         | 43.6MB 35.6MB/s eta 0:00:05    20% |██████▋                         | 44.9MB 31.0MB/s eta 0:00:06    22% |███████                         | 47.9MB 12.7MB/s eta 0:00:14    22% |███████▏                        | 48.9MB 13.0MB/s eta 0:00:14    22% |███████▍                        | 50.0MB 13.4MB/s eta 0:00:13    29% |█████████▌                      | 64.8MB 12.1MB/s eta 0:00:13    31% |██████████       

In [1]:
#pyspark initalization and connection

import findspark
findspark.init()

In [2]:
findspark.init("/home/ubuntu/server/spark-2.4.5-bin-hadoop2.7/")
from pyspark import SparkContext, SparkConf

In [3]:
# create Spark context with Spark configuration

from pyspark.sql import SQLContext

conf = SparkConf().setAppName("Spark Count1")
sc = SparkContext(conf=conf)

sqlContext = SQLContext(sc)

In [9]:
# Loaded tweeter and newsapi data into python list from mongodb

li = []
s = tweet_collection.find({})
for i in s:
    li.append(i['text'])

s = news_collection.find({})
for i in s:
    li.append(i['description'])

In [10]:
#creating spark dataframe

from pyspark.sql import Row

rdd1 = sc.parallelize(li)

row_rdd = rdd1.map(lambda x: Row(x))

df_my = sqlContext.createDataFrame(row_rdd,['numbers'])
shakeWordsDFCount = df_my.count()
print (shakeWordsDFCount)

3619


In [13]:
#word count function

s = df_my.withColumn('word', f.explode(f.split(f.col('numbers'), ' ')))\
    .groupBy('word')\
    .count()\
    .sort('count', ascending=False)

In [21]:
import json
results = s.toJSON().map(lambda j: json.loads(j)).collect()

In [26]:
search_keywords = ["education","canada","university","dalhousie","expensive","good school",
                   "good schools","bad school","bad schools","poor school","poor schools",
                   "faculty","computer science","graduate"]

for i in results: 
    if i["word"] in search_keywords:
        print(str(i['word']) + " --> " + str(i['count']))

canada --> 1222
university --> 641
education --> 386
dalhousie --> 148
graduate --> 6
expensive --> 6
faculty --> 3


In [ ]:
#This are the keywords and their count and rest of them have 0 count.